In [74]:
setprecision(256)

using Plots

plotly()

C = 3000000

3000000

## Generator testów

In [75]:
function gen_test(satellites)
    # wektor [x; y; z; t] - (x, y, z): nasza pozycja, t: błąd zegara
    X = vcat(Array{BigFloat}(2000 * rand(3) - 1000) .^ 3, Array{BigFloat}(100 * randn(1)) .^ 3)
    data = Array{BigFloat, 2}(4, satellites)
    for i = 1:satellites
        v = Array{BigFloat}(2000 * rand(3) - 1000) .^ 3
        data[:, i] = X + vcat(v, norm(v) / C)
    end
    return data, X
end

gen_test (generic function with 1 method)

## Dane do testów

In [76]:
data, solution₁ = gen_test(4)

# data[1, :] : wektor współrzędnych x satelit
# data[2, :] : wektor współrzędnych y satelit
# data[3, :] : wektor współrzędnych z satelit
# data[4, :] : wektor czasów zegara satelit

# data[:, i] : dane z i-tej satelity
# solution₁ = poszukiwane rozwiązanie (niekoniecznie jedyne)

# Funkcja F jak w sprawozdaniu
function f(x::Array{BigFloat})
    return ((-data .+ x) .^ 2)' * [1, 1, 1, -C*C]
end


JacobianArray = Array{Function, 2}(4, 4)
for i = 1:4
    for j = 1:3
        JacobianArray[i, j] = function(x) return 2*x[j] - 2*data[j, i] end
    end
    JacobianArray[i, 4] = function(x) return 2*C*C*data[4, i] - 2*C*C*x[4] end
end

# Jakobian jak w sprawozdaniu
function Jacobian(x::Array{BigFloat})
    return map((f) -> f(x), JacobianArray)
end


Jacobian (generic function with 1 method)

# Metoda Newtona

In [77]:
function prnt(a::Array{Float32})
    @printf "%.4e, %.4e, %.4e, %.4e" a[1] a[2] a[3] a[4]
end

function NewtonMethod(x::Array{BigFloat}, ϵ)
    while true 
        if norm(f(x)) < ϵ
            return x
        end
        @printf "x - sol: ["
        prnt(abs.(Array{Float32}(x - solution₁)))
        @printf "]\tf(x) : ["
        prnt(abs.(Array{Float32}(f(x))))
        @printf "]\n"
        δ = Jacobian(x) \ f(x)
        x = x - δ
    end
end

NewtonMethod(Array{BigFloat}([1, 1, 1, 0]), 1e-50)

x - sol: [3.9074e+05, 2.2120e+06, 5.9191e+07, 7.4325e+04]	f(x) : [5.0073e+22, 5.0004e+22, 5.0128e+22, 5.0147e+22]
x - sol: [2.9216e+10, 2.1603e+10, 2.8957e+10, 3.7048e+04]	f(x) : [1.0351e+22, 1.0351e+22, 1.0351e+22, 1.0351e+22]
x - sol: [1.4497e+10, 1.0719e+10, 1.4368e+10, 1.8383e+04]	f(x) : [2.5876e+21, 2.5876e+21, 2.5876e+21, 2.5876e+21]
x - sol: [7.1378e+09, 5.2779e+09, 7.0747e+09, 9.0514e+03]	f(x) : [6.4674e+20, 6.4674e+20, 6.4674e+20, 6.4674e+20]
x - sol: [3.4601e+09, 2.5585e+09, 3.4295e+09, 4.3878e+03]	f(x) : [1.6154e+20, 1.6154e+20, 1.6154e+20, 1.6154e+20]
x - sol: [1.6247e+09, 1.2014e+09, 1.6104e+09, 2.0603e+03]	f(x) : [4.0234e+19, 4.0234e+19, 4.0234e+19, 4.0234e+19]
x - sol: [7.1379e+08, 5.2779e+08, 7.0747e+08, 9.0514e+02]	f(x) : [9.9104e+18, 9.9104e+18, 9.9104e+18, 9.9104e+18]
x - sol: [2.7154e+08, 2.0078e+08, 2.6914e+08, 3.4433e+02]	f(x) : [2.3359e+18, 2.3359e+18, 2.3359e+18, 2.3359e+18]
x - sol: [7.4341e+07, 5.4970e+07, 7.3684e+07, 9.4271e+01]	f(x) : [4.6443e+17, 4.6443e+17

4-element Array{BigFloat,1}:
 -3.907348079175006011381193324311854523263299965623735637006602033426978734971857e+05
  2.211961307955054052807633967881075144161299690343898478400668109103433920874838e+06
  5.919129435661394036744584354848745327951217642935148871532112208992216146104164e+07
  7.432511224252961855774193332908852244133311163616420947980583675726161285627736e+04

In [2]:
# Metoda algebraiczna